*Training an XGBoost model on the Walmart Daily data to identify anomalies in it.*

# Start

## Import

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib
import datetime 

In [2]:
base_folder = 'C:\\Geeta\\learning\\projects\\AnomalyDetectionSXM\\Notebooks\\Datasets\\Pipeline'
dataset_name = 'Walmart_Daily'
train_file = base_folder + '/train/' + dataset_name +'_train.csv'
inference_file = base_folder + '/inference/' + dataset_name +'_inference.csv'

current_date_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_file_path = base_folder + '/xgboostADmodel_' + dataset_name + current_date_time + '.pkl'

inference_results_file = base_folder + '/inference/' + dataset_name +'_inference_results.csv'

# target = 'Weekly_Sales'
target = 'Daily_Sales'

# Group the data by 'State' and perform lag shifting within each group
groupby_cols=['State'] 

# Define lag columns to consider
lag_columns = [target, 'Temperature']#, 'Fuel_Price', 'CPI', 'Unemployment']

# Define the lag values to be used
lags = [1, 2, 4]  # Example lag values of 1 week and 2 weeks

state_encoder = 'label_encoder_State_daily.pkl'

In [3]:
data_path = train_file

In [4]:
data_path

'C:\\Geeta\\learning\\projects\\AnomalyDetectionSXM\\Notebooks\\Datasets\\Pipeline/train/Walmart_Daily_train.csv'

## Load Dataset

In [5]:
def read(data_path, sheet_name = '', usecols = None):
    df = pd.DataFrame()
    if data_path.split('.')[-1] == 'xlsx':
        if sheet_name:
            df = pd.read_excel(data_path, sheet_name=sheet_name, usecols=usecols)
        else:
            df = pd.read_excel(data_path, usecols=usecols)
        print("Shape of the data in file {} is {}".format(data_path, df.shape))
    else:
        try:
            df = pd.read_csv(data_path)
            print("Shape of the data in file {} is {}".format(data_path, df.shape))
            if df.shape[0] == 0:
                print("No data in file {}".format(data_path))
        except Exception as e:
            print("Issue while reading data at {} \n{}".format(data_path, e))
    return df


def standardize_date_col(dataframe, date_col):
    dataframe[date_col] = pd.to_datetime(dataframe[date_col], format='%Y-%m-%d')#, errors='coerce').fillna(pd.to_datetime(dataframe['Date'], format='%d/%m/%y', errors='coerce'))
    # # Convert all dates to 'mm-dd-yyyy' format
    # dataframe[date_col] = dataframe[date_col].dt.strftime('%Y-%m-%d')
    return dataframe

     

In [6]:
# Read data from csv or excel, sheet_name is the sheet in excel that contians data 
data = read(data_path, sheet_name= 'RAW')
data = standardize_date_col(data, 'Date')
data.head(3)

Shape of the data in file C:\Geeta\learning\projects\AnomalyDetectionSXM\Notebooks\Datasets\Pipeline/train/Walmart_Daily_train.csv is (4990, 11)


,Date,Daily_Sales,Temperature,Fuel_Price,CPI,Unemployment,Holiday_Flag,Anomaly,Sales_Amount_Upper,Sales_Amount_Lower,State
0,2021-02-01,1610296.53,32.53,2.77,150.5,8.84,0,0,3076267.48,1029590.71,California
1,2021-02-02,2896681.32,32.53,2.77,150.5,8.84,0,0,2975331.69,928654.93,California
2,2021-02-03,1645907.96,32.53,2.77,150.5,8.84,0,0,3035540.67,988863.91,California


In [7]:
# Sort the data by the 'Date' column in ascending order
data = data.sort_values('Date')

In [8]:
data.columns 

Index(['Date', 'Daily_Sales', 'Temperature', 'Fuel_Price', 'CPI',
       'Unemployment', 'Holiday_Flag', 'Anomaly', 'Sales_Amount_Upper',
       'Sales_Amount_Lower', 'State'],
      dtype='object')

## Preprocessing data

In [9]:
def get_lag_columns(data, groupby_cols, lag_columns):
    # Group the data by 'State' and perform lag shifting within each group
    grouped = data.groupby(groupby_cols)
    
    
    # Create lag features within each group
    for lag in lags:
        for col in lag_columns:
            data[f'{col}_lag_{lag}'] = grouped[col].shift(lag)
            data[f'{col}_lag_{lag}'] = data[f'{col}_lag_{lag}'].bfill()
    return data

data = get_lag_columns(data, groupby_cols, lag_columns)

In [10]:
data.columns

Index(['Date', 'Daily_Sales', 'Temperature', 'Fuel_Price', 'CPI',
       'Unemployment', 'Holiday_Flag', 'Anomaly', 'Sales_Amount_Upper',
       'Sales_Amount_Lower', 'State', 'Daily_Sales_lag_1', 'Temperature_lag_1',
       'Daily_Sales_lag_2', 'Temperature_lag_2', 'Daily_Sales_lag_4',
       'Temperature_lag_4'],
      dtype='object')

In [11]:
def get_date_features(data, date_col):
    def week_of_month(date):
        # Get the first day of the month
        first_day = date.replace(day=1)
        # Calculate the adjusted day of the week (0=Monday, ..., 6=Sunday)
        adjusted_dom = (first_day.weekday() + 1) % 7
        # Calculate the week of the month
        week_of_month = (date.day + adjusted_dom - 1) // 7 + 1
        return week_of_month
    
    data['Week_Of_Month'] = data[date_col].map(week_of_month)
    # Create time-based features
    data[date_col] = pd.to_datetime(data[date_col])
    data['Day_of_Week'] = data['Date'].dt.dayofweek  # Day of the week (0: Monday, 1: Tuesday, ..., 6: Sunday)
    data['Month'] = data[date_col].dt.month  # Month of the year (1 to 12)
    data['Quarter'] = data[date_col].dt.quarter  # Quarter of the year (1 to 4)
    data['Year'] = data[date_col].dt.year  # Year
    return data

data = get_date_features(data, 'Date')
data.tail(5)

,Date,Daily_Sales,Temperature,Fuel_Price,CPI,Unemployment,Holiday_Flag,Anomaly,Sales_Amount_Upper,Sales_Amount_Lower,...,Temperature_lag_1,Daily_Sales_lag_2,Temperature_lag_2,Daily_Sales_lag_4,Temperature_lag_4,Week_Of_Month,Day_of_Week,Month,Quarter,Year
3991,2023-10-26,1555295.33,64.08,3.79,171.40,7.76,0,0,1676746.33,305022.24,...,64.09,564663.16,64.09,1477457.32,64.09,4,3,10,4,2023
1995,2023-10-26,1749899.53,66.46,3.53,211.93,5.67,0,0,2182135.06,242546.46,...,66.45,1530906.25,66.45,1797874.67,66.45,4,3,10,4,2023
997,2023-10-26,1343070.18,56.59,3.91,157.86,7.23,0,0,2967295.81,920619.04,...,56.59,1116258.44,56.59,1197744.02,56.59,4,3,10,4,2023
2993,2023-10-26,1958648.00,56.99,3.92,166.21,7.14,0,0,2571818.50,529280.99,...,56.99,1559417.49,57.00,1843485.91,57.00,4,3,10,4,2023
4989,2023-10-26,845685.88,61.13,3.82,175.88,6.96,0,0,1725018.24,463462.65,...,61.13,1061565.59,61.13,817229.78,61.13,4,3,10,4,2023


In [12]:
# Encode categorical columns
encoder = LabelEncoder()
data['State'] = encoder.fit_transform(data['State'])

# Save the trained label encoder
joblib.dump(encoder, state_encoder)

# Split the data into features and target variable
X = data.drop(['Date', 'Anomaly', 'Sales_Amount_Upper', 'Sales_Amount_Lower'], axis=1)
y = data['Anomaly']

# Perform train-test split (80:20 ratio)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Display the shapes of the datasets after splitting
X_train.shape, X_val.shape, X_test.shape


((3193, 18), (799, 18), (998, 18))

In [13]:
X_train.head(3)

,Daily_Sales,Temperature,Fuel_Price,CPI,Unemployment,Holiday_Flag,State,Daily_Sales_lag_1,Temperature_lag_1,Daily_Sales_lag_2,Temperature_lag_2,Daily_Sales_lag_4,Temperature_lag_4,Week_Of_Month,Day_of_Week,Month,Quarter,Year
3750,1190813.87,50.00,3.71,169.80,8.53,0,3,1684568.78,50.00,756580.64,48.71,738881.69,48.71,5,0,2,1,2023
4335,953596.87,34.17,3.09,168.10,8.67,0,4,1227654.07,34.17,869446.63,36.38,668521.93,36.38,3,0,1,1,2022
4611,621400.76,63.39,3.50,172.21,8.01,0,4,1712507.88,63.39,626702.40,63.39,1228146.37,63.39,3,3,10,4,2022


In [14]:
X_train.columns

Index(['Daily_Sales', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment',
       'Holiday_Flag', 'State', 'Daily_Sales_lag_1', 'Temperature_lag_1',
       'Daily_Sales_lag_2', 'Temperature_lag_2', 'Daily_Sales_lag_4',
       'Temperature_lag_4', 'Week_Of_Month', 'Day_of_Week', 'Month', 'Quarter',
       'Year'],
      dtype='object')

## Training model

In [15]:
# Define a narrower set of hyperparameters
params = {
    'n_estimators': 100,
    'max_depth': 5,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

# Train the XGBoost model with the reduced set of hyperparameters
model = XGBClassifier(**params, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the validation set
y_val_pred = model.predict(X_val)

# Evaluate the model performance on the validation set
validation_accuracy = accuracy_score(y_val, y_val_pred)
validation_report = classification_report(y_val, y_val_pred)

print("Validation Accuracy:", validation_accuracy)
print("Validation Report:")
print(validation_report)

# Make predictions on the test set
y_test_pred = model.predict(X_test)


Validation Accuracy: 0.9749687108886108
Validation Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       762
           1       0.90      0.51      0.66        37

    accuracy                           0.97       799
   macro avg       0.94      0.76      0.82       799
weighted avg       0.97      0.97      0.97       799



## Saving model

In [16]:
model_file_path

'C:\\Geeta\\learning\\projects\\AnomalyDetectionSXM\\Notebooks\\Datasets\\Pipeline/xgboostADmodel_Walmart_Daily2024-04-20_21-42-44.pkl'

In [17]:
# Save the trained model to a file
joblib.dump(model, model_file_path)

['C:\\Geeta\\learning\\projects\\AnomalyDetectionSXM\\Notebooks\\Datasets\\Pipeline/xgboostADmodel_Walmart_Daily2024-04-20_21-42-44.pkl']

## Load Model

In [18]:
# Load the saved model from file
loaded_model = joblib.load(model_file_path)


## Validation

In [19]:
# Evaluate the model performance on the test set
test_accuracy = accuracy_score(y_test, y_test_pred)
test_report = classification_report(y_test, y_test_pred)

print("\nTest Accuracy:", test_accuracy)
print("Test Report:")
print(test_report)


Test Accuracy: 0.9639278557114228
Test Report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       948
           1       0.77      0.40      0.53        50

    accuracy                           0.96       998
   macro avg       0.87      0.70      0.75       998
weighted avg       0.96      0.96      0.96       998



## Inference

In [20]:
# Load the new dataset for inference
inference_data = pd.read_csv(inference_file, usecols=['Date',target, 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment',
       'Holiday_Flag','State'])

inference_data = standardize_date_col(inference_data, 'Date')

# Convert the 'Date' column to datetime format
# inference_data['Date'] = pd.to_datetime(inference_data['Date']).dt.strftime('%Y-%m-%d')

# Get the saved Encoder for State column 
encoder = joblib.load(state_encoder)
inference_data['State'] = encoder.fit_transform(inference_data['State'])

In [21]:
data['Date'].max(), inference_data['Date'].min(), inference_data['Date'].isna().sum()


(Timestamp('2023-10-26 00:00:00'), Timestamp('2023-10-27 00:00:00'), 0)

In [22]:
cols = inference_data.columns

In [23]:
inference_data_all = pd.concat([data[cols],inference_data])
inference_data_all

,Date,Daily_Sales,Temperature,Fuel_Price,CPI,Unemployment,Holiday_Flag,State
0,2021-02-01,1610296.53,32.53,2.77,150.50,8.84,0,0
2994,2021-02-01,647356.29,39.26,2.71,163.00,9.68,0,3
1996,2021-02-01,1085410.27,22.87,2.82,157.34,8.03,0,2
3992,2021-02-01,645338.48,38.33,2.72,167.21,8.99,0,4
998,2021-02-01,1044395.77,37.20,2.57,200.61,7.55,0,1
...,...,...,...,...,...,...,...,...
0,2023-10-27,2430803.18,56.59,3.91,157.86,7.23,0,0
1,2023-10-27,1022090.78,66.46,3.53,211.92,5.67,0,1
2,2023-10-27,1215995.54,57.00,3.92,166.20,7.14,0,2
3,2023-10-27,932135.59,64.09,3.79,171.40,7.76,0,3


In [24]:

# Sort the data by the 'Date' column in ascending order
inference_data_all = inference_data_all.sort_values('Date')

# Create lag features for the relevant columns
for lag in lags:
    for col in lag_columns:
        inference_data_all[f'{col}_lag_{lag}'] = inference_data_all[col].shift(lag)

inference_data_all

,Date,Daily_Sales,Temperature,Fuel_Price,CPI,Unemployment,Holiday_Flag,State,Daily_Sales_lag_1,Temperature_lag_1,Daily_Sales_lag_2,Temperature_lag_2,Daily_Sales_lag_4,Temperature_lag_4
0,2021-02-01,1610296.53,32.53,2.77,150.50,8.84,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2994,2021-02-01,647356.29,39.26,2.71,163.00,9.68,0,3,1610296.53,32.53,NaN,NaN,NaN,NaN
1996,2021-02-01,1085410.27,22.87,2.82,157.34,8.03,0,2,647356.29,39.26,1610296.53,32.53,NaN,NaN
3992,2021-02-01,645338.48,38.33,2.72,167.21,8.99,0,4,1085410.27,22.87,647356.29,39.26,NaN,NaN
998,2021-02-01,1044395.77,37.20,2.57,200.61,7.55,0,1,645338.48,38.33,1085410.27,22.87,1610296.53,32.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,2023-10-27,1215995.54,57.00,3.92,166.20,7.14,0,2,1958648.00,56.99,1343070.18,56.59,1555295.33,64.08
3,2023-10-27,932135.59,64.09,3.79,171.40,7.76,0,3,1215995.54,57.00,1958648.00,56.99,1749899.53,66.46
0,2023-10-27,2430803.18,56.59,3.91,157.86,7.23,0,0,932135.59,64.09,1215995.54,57.00,1343070.18,56.59
1,2023-10-27,1022090.78,66.46,3.53,211.92,5.67,0,1,2430803.18,56.59,932135.59,64.09,1958648.00,56.99


In [25]:
inference_data_all[inference_data_all['Date']=='2023-10-27']

,Date,Daily_Sales,Temperature,Fuel_Price,CPI,Unemployment,Holiday_Flag,State,Daily_Sales_lag_1,Temperature_lag_1,Daily_Sales_lag_2,Temperature_lag_2,Daily_Sales_lag_4,Temperature_lag_4
2,2023-10-27,1215995.54,57.00,3.92,166.20,7.14,0,2,1958648.00,56.99,1343070.18,56.59,1555295.33,64.08
3,2023-10-27,932135.59,64.09,3.79,171.40,7.76,0,3,1215995.54,57.00,1958648.00,56.99,1749899.53,66.46
0,2023-10-27,2430803.18,56.59,3.91,157.86,7.23,0,0,932135.59,64.09,1215995.54,57.00,1343070.18,56.59
1,2023-10-27,1022090.78,66.46,3.53,211.92,5.67,0,1,2430803.18,56.59,932135.59,64.09,1958648.00,56.99
4,2023-10-27,1352862.94,61.13,3.82,175.87,6.96,0,4,1022090.78,66.46,2430803.18,56.59,1215995.54,57.00


In [26]:
# Get the inference data back with lag values:
inference_data = inference_data_all[inference_data_all['Date']=='2023-10-27']
# Get week of the month value too before passing it to model
inference_data = get_date_features(inference_data, 'Date')
inference_data.head()

C:\Users\xyz\AppData\Local\Temp\ipykernel_24636\1430584753.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Week_Of_Month'] = data[date_col].map(week_of_month)
C:\Users\xyz\AppData\Local\Temp\ipykernel_24636\1430584753.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[date_col] = pd.to_datetime(data[date_col])
C:\Users\xyz\AppData\Local\Temp\ipykernel_24636\1430584753.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

,Date,Daily_Sales,Temperature,Fuel_Price,CPI,Unemployment,Holiday_Flag,State,Daily_Sales_lag_1,Temperature_lag_1,Daily_Sales_lag_2,Temperature_lag_2,Daily_Sales_lag_4,Temperature_lag_4,Week_Of_Month,Day_of_Week,Month,Quarter,Year
2,2023-10-27,1215995.54,57.00,3.92,166.20,7.14,0,2,1958648.00,56.99,1343070.18,56.59,1555295.33,64.08,4,4,10,4,2023
3,2023-10-27,932135.59,64.09,3.79,171.40,7.76,0,3,1215995.54,57.00,1958648.00,56.99,1749899.53,66.46,4,4,10,4,2023
0,2023-10-27,2430803.18,56.59,3.91,157.86,7.23,0,0,932135.59,64.09,1215995.54,57.00,1343070.18,56.59,4,4,10,4,2023
1,2023-10-27,1022090.78,66.46,3.53,211.92,5.67,0,1,2430803.18,56.59,932135.59,64.09,1958648.00,56.99,4,4,10,4,2023
4,2023-10-27,1352862.94,61.13,3.82,175.87,6.96,0,4,1022090.78,66.46,2430803.18,56.59,1215995.54,57.00,4,4,10,4,2023


In [27]:

inference_data['Daily_Sales'].iloc[0]

1215995.54

In [28]:
# 1112863943

inference_data['Daily_Sales'].iloc[0] = 121599

C:\Users\xyz\AppData\Local\Temp\ipykernel_24636\2573748765.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  inference_data['Daily_Sales'].iloc[0] = 121599
C:\Users\xyz\AppData\Local\Temp\ipykernel_24636\2573748765.py:3: SettingWithCopyWar

In [29]:
inference_data.iloc[0]

Date                 2023-10-27 00:00:00
Daily_Sales                     121599.0
Temperature                         57.0
Fuel_Price                          3.92
CPI                                166.2
Unemployment                        7.14
Holiday_Flag                           0
State                                  2
Daily_Sales_lag_1              1958648.0
Temperature_lag_1                  56.99
Daily_Sales_lag_2             1343070.18
Temperature_lag_2                  56.59
Daily_Sales_lag_4             1555295.33
Temperature_lag_4                  64.08
Week_Of_Month                          4
Day_of_Week                            4
Month                                 10
Quarter                                4
Year                                2023
Name: 2, dtype: object

In [30]:
inference_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 2 to 4
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               5 non-null      datetime64[ns]
 1   Daily_Sales        5 non-null      float64       
 2   Temperature        5 non-null      float64       
 3   Fuel_Price         5 non-null      float64       
 4   CPI                5 non-null      float64       
 5   Unemployment       5 non-null      float64       
 6   Holiday_Flag       5 non-null      int64         
 7   State              5 non-null      int32         
 8   Daily_Sales_lag_1  5 non-null      float64       
 9   Temperature_lag_1  5 non-null      float64       
 10  Daily_Sales_lag_2  5 non-null      float64       
 11  Temperature_lag_2  5 non-null      float64       
 12  Daily_Sales_lag_4  5 non-null      float64       
 13  Temperature_lag_4  5 non-null      float64       
 14  Week_Of_Month      

In [31]:
# Perform inference using the loaded model
predictions = loaded_model.predict(inference_data.drop(columns=['Date']))

# Display the predictions
predictions

array([0, 0, 0, 0, 0])

## Saving inference

# End